In [36]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, MetaData, Table
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
import sqlite3

In [56]:
df = pd.read_csv("Data/total_data.csv")
df= df.reset_index(drop=False)
df = df.set_index('index')
df

,country,rank,score,economy,family,health,freedom,generosity,trust,year,lat,long
index,,,,,,,,,,,,
0,Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2015,46.818188,8.227512
1,Iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2015,64.963051,-19.020835
2,Denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2015,56.263920,9.501785
3,Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2015,60.472024,8.468946
4,Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2015,56.130366,-106.346771
...,...,...,...,...,...,...,...,...,...,...,...,...
776,Rwanda,152,3.334,0.35900,0.71100,0.61400,0.55500,0.21700,0.41100,2019,-1.940278,29.873888
777,Tanzania,153,3.231,0.47600,0.88500,0.49900,0.41700,0.27600,0.14700,2019,-6.369028,34.888822
778,Afghanistan,154,3.203,0.35000,0.51700,0.36100,0.00000,0.15800,0.02500,2019,33.939110,67.709953


In [58]:
conn = sqlite3.connect('Data/happiness.sqlite')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

# Create table - CLIENTS
c.execute('''DROP TABLE happiness''')
c.execute('''
             CREATE TABLE happiness
             (
             [index] integer primary key,
             [country] text, 
             [rank] integer, 
             [score] real,
             [economy] real,
             [family] real,
             [health] real,
             [generosity] real,
             [freedom] real,
             [trust] real,
             [year] integer,
             [lat] real,
             [long] real
             )''')


In [59]:
conn.commit()

In [31]:
df.to_sql('happiness', conn, if_exists='append', index = False)

In [ ]:
#df.to_sql(User.__tablename__, conn, index=False, if_exists="append")

In [60]:
engine = create_engine("sqlite:///Data/happiness.sqlite")
con = engine.connect()

In [61]:
Base = declarative_base()

# class Empty(Base):
#     __tablename__ = "blank" # required field
#     index = Column(Integer, primary_key=True, nullable=False, autoincrement=False)
    
class User(Base):
    __tablename__ = "happiness" # required field  
    index = Column(Integer, primary_key=True, nullable=False)
    country = Column(String(100))
    rank = Column(Integer)
    score = Column(Float)
    economy = Column(Float)
    family = Column(Float)
    health = Column(Float)
    freedom = Column(Float)
    generosity = Column(Float)
    trust = Column(Float)
    year = Column(Integer)
    lat = Column(Float)
    long = Column(Float)

In [62]:
df.to_sql(User.__tablename__, con, index=False, if_exists="append")

In [66]:
# reflect an existing database into a new model
Base = automap_base()
# # reflect the tables
Base.prepare(engine, reflect=True)
#classes
Base.classes.keys()
happiness = Base.classes.happiness

In [67]:
session = Session(engine)

In [69]:
results = session.query(happiness.country, happiness.rank, happiness.score, happiness.economy, happiness.family, happiness.health, happiness.freedom, happiness.generosity, happiness.trust, happiness.year, happiness.lat, happiness.long).all()
data_dict = {}
data_list = []
for country, rank, score, economy, family, health, freedom, generosity, trust, year, lat, long in results:
    data_dict['Country'] = country
    data_dict['Rank'] = rank
    data_dict['Score']= score
    data_dict['Economy']= economy
    data_dict['Family']= family
    data_dict['Health']= health
    data_dict['Freedom']= freedom
    data_dict['Generosity']= generosity
    data_dict['Trust']= trust
    data_dict['Year']= year
    data_dict['Lat']= lat
    data_dict['Long']= long
    data_list.append(data_dict)
    data_dict = {}